# XFEAt Screw dislocation

Create a screw dislocation in a bcc Fe crystal and apply a shear stress until the
dislocation starts moving.

Author: Alexander Hartmaier  
Institution: ICAMS / Ruhr University Bochum, Germany  
Date: March 2022

This work is licensed under a
Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License
<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">(CC-BY-NC-SA)</a>

The XFEAt package comes with ABSOLUTELY NO WARRANTY. This is free
software, and you are welcome to redistribute it under the conditions of
the GNU General Public License <a href="http://www.fsf.org/licensing/licenses/gpl.html">(GPLv3)</a>

<img alt="Creative Commons License" style="border-width:0;max-heigt:9px;height:100%;" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png"/></a>
<img alt="GPLv3" style="border-width:0;max-heigt:9px;height:100%;" src="gplv3-88x31.png"/></a>

In [1]:
%matplotlib inline
from pyvista import set_plot_theme, set_jupyter_backend
set_plot_theme('document')
set_jupyter_backend('ipyvtklink')

## 1. Create model

Define a material and create an XFEAt model consisting of an atomistic core and an XFEM frame around it. Insert a screw disloction into the XFEM and atomistic parts of the model.

In [2]:
import xfeat

# define material of single crystal as dictionary
mat = {
       'name' : 'iron',
       'cs'   : 'bcc',  # crystal structure
       'lp'   : 2.8553,  # lattice parameter in Angstrom
       'mass' : 55.845,  # atomic mass in u
       # define anisotropic elastic constants in GPa
       'C11'  : 243.4,
       'C12'  : 145.0,
       'C44'  : 116.0,
       # W-Bop 6
       # C11. C12, C44 = 2.837825, 1.5317, 0.6256225
       # define crystallograhic orientation of crystal
       'ori_x' : [-1, 2, -1],
       'ori_y' : [-1, 0,  1],
       'ori_z' : [ 1, 1,  1],
       }

# create XFEM model 
mod = xfeat.Model(mat, size=400)
# create atomic core
mod.atoms([10, 17, 3])
mod.atom_grid.plot(cpos='xy', style='points', point_size=20, render_points_as_spheres=True)
# create mesh and set up system stiffness matrix
mod.mesh()
mod.grid.plot(cpos='xy', show_edges=True)
# create screw dislocation
mod.init_dislo([0, 0, 1])
# extract atomistic displacements
mod.atom_bc()
# plot z-displaceent of atoms and XFEM nodes
mod.plot('uz')

Succesfully created structure with 6120 atoms.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Generated mesh with 17376 elements and 23872 nodes around atomistic core.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in 13.452s.
Relaxation of atoms obtained in 37.085s.

 Created dislocation with Burgers vector ([0. 0. 1.]) in model.
Norm of Burgers vector is 2.472772 A


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

It is seen that the z-displacement in atomistic and XFEM regions are constistently defined when the screw dislocation is introduced. However, the other displacement components are depending of the interatomic potential and cannot be set directly.

In [4]:
# plot y-displacements
mod.plot('uy')
print('Zoom in to see atoms lying in overlap region more clearly.')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Zoom in to see atoms lying in overlap region more clearly.


To embedd the dislocation consistently into the XFEM model, it is necessary to relax both structures iteratively. During this process, the atoms overlapping with the XFEM region are always fixed to the XFEM solution, whereas the atoms in the inner region are relaxed freely. After the atomic relaxation has occured, the displacements of the relaxed atoms on the boundary of the inner region are applied as XFEM boundary conditions and a new XFEM solution is calculated.

In [5]:
# iterate into relaxed configuration
for i in range(12):
    mod.atom_bc()  # apply relaxed atom positions as BC to XFEM 
    mod.solve()  # colculate nodal displacements for mechanical equilibrium
    mod.shift_atoms()  # move boundary atoms in overlap region according to XFEM strain field
    mod.relax_atoms(i)  # relax atomic structure with fixed boundary atoms
    if i % 3 == 2:
        print(f'y-displacement after {i+1} iterations. Zoom in to overlap region.')
        mod.plot('uy')

XFEM solution obtained in 12.269s.
Relaxation of atoms obtained in 43.512s.
XFEM solution obtained in 12.393s.
Relaxation of atoms obtained in 39.973s.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in 12.936s.
Relaxation of atoms obtained in 42.935s.
XFEM solution obtained in 12.341s.
Relaxation of atoms obtained in 42.261s.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in 13.602s.
Relaxation of atoms obtained in 38.372s.
XFEM solution obtained in 12.367s.
Relaxation of atoms obtained in 40.818s.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in 13.176s.
Relaxation of atoms obtained in 38.059s.
XFEM solution obtained in 12.722s.
Relaxation of atoms obtained in  36.76s.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in 12.908s.
Relaxation of atoms obtained in 38.577s.
XFEM solution obtained in 13.249s.
Relaxation of atoms obtained in 37.215s.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## 2. Apply shear stress
In a first step, a sub-critical shear stress is applied at the XFEM boundary, which is not sufficient to move the dislocation.

In [10]:
# plot nodal displacement
print('z-displacement before applying shear stress on XFEM model.')
mod.plot('uz')
# plot stresses
print('Stress component yz before applying shear stress.')
mod.plot('sigyz')
# plot potential energy of atoms
print('Potential energy of atoms before applying shear stress.')
mod.plot('epot')

# Apply sub-critical shear stress on boundary
val = 1.55
mod.apply_bc(val, bc_type='stress', comp='yz')
print(f'Shear stress of {val} GPa applied on XFEM boundary.')
# iterate into relaxed configuration
for i in range(10):
    mod.atom_bc()
    mod.solve()
    mod.shift_atoms()
    mod.relax_atoms(i, name='applied_stress_155')
    if i % 5 == 4:
        print(f'Potential energy of atoms after {i+1} iterations.')
        mod.plot('epot')
# plot stresses
print(f'Stress component yz at applied shear stress {val} GPa.')
mod.plot('sigyz')

z-displacement before applying shear stress on XFEM model.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Stress component yz before applying shear stress.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Potential energy of atoms before applying shear stress.


...

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in 13.684s.
Relaxation of atoms obtained in 49.723s.
XFEM solution obtained in 13.096s.
Relaxation of atoms obtained in  45.17s.
XFEM solution obtained in 13.179s.
Relaxation of atoms obtained in 45.741s.
XFEM solution obtained in 13.178s.
Relaxation of atoms obtained in 40.477s.
XFEM solution obtained in  13.72s.
Relaxation of atoms obtained in 37.997s.
Potential energy of atoms after 10 iterations.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Stress component yz after applying shear stress.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## 3. Increase stress 
Now, the level of the applied shear stress is increased beyond the critical value and dislocation motion sets in.

In [12]:
# Apply critical shear stress on boundary
val = 1.65
mod.apply_bc(val, bc_type='stress', comp='yz')
# iterate into relaxed configuration
for i in range(10):
    mod.atom_bc()
    mod.solve()
    mod.shift_atoms()
    mod.relax_atoms(i, name='applied_stress_165')
    if i % 5 == 4:
        print(f'Potential energy of atoms after {i+1} iterations.')
        mod.plot('epot')

# plot stresses
print(f'Stress component yz at applied shear stress {val} GPa.')
mod.plot('sigyz')


XFEM solution obtained in 12.353s.
Relaxation of atoms obtained in 46.973s.
XFEM solution obtained in 12.368s.
Relaxation of atoms obtained in 45.308s.
XFEM solution obtained in 12.338s.
Relaxation of atoms obtained in 49.238s.
XFEM solution obtained in 12.301s.
Relaxation of atoms obtained in 46.068s.
XFEM solution obtained in 12.451s.
Relaxation of atoms obtained in 45.178s.
Potential energy of atoms after 5 iterations.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

XFEM solution obtained in 12.357s.
Relaxation of atoms obtained in 45.798s.
XFEM solution obtained in 15.019s.
Relaxation of atoms obtained in 85.681s.
XFEM solution obtained in 12.382s.
Relaxation of atoms obtained in 82.683s.
XFEM solution obtained in 12.339s.
Relaxation of atoms obtained in 85.197s.
XFEM solution obtained in 12.415s.
Relaxation of atoms obtained in 36.706s.
Potential energy of atoms after 10 iterations.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

Stress component yz at applied shear stress 1.65 GPa.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

It is seen that the dislocation starts moving at this stress level, which also increases the yz-stress component at the rhs of the inner boundary of the XFEM disproportionately.